In [ ]:
# !pip install ollama
# !pip install tqdm
#pip install langchain-community
#pip install gputil

In [3]:
import psutil
import GPUtil
import time
import subprocess
from tqdm import tqdm 

def get_gpu_name():
    gpus = GPUtil.getGPUs()
    return gpus[0].name if gpus else "Unknown_GPU"


def get_cpu_name():
    cmd = 'wmic cpu get Name'
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    output = proc.communicate()[0].decode('utf-8').strip()
    lines = [line.strip() for line in output.splitlines() if line.strip()]
    cpu_name_line = lines[1] if len(lines) > 1 else "Unknown_CPU"

    # 提取 CPU 型號
    cpu_name, cpu_model = "Unknown_CPU", "Unknown_Model"
    cpu_name_parts = cpu_name_line.split()
    for part in cpu_name_parts:
        if "i7" in part or "i9" in part or "i5" in part:
            cpu_name = part.split('-')[0]
            cpu_model = part.split('-')[-1]
            break
    if cpu_model == "Unknown_Model":
        for part in cpu_name_parts:
            if any(char.isdigit() for char in part):
                cpu_model = part
                break
    return f"{cpu_name}_{cpu_model}"


def log_iteration_results(i, execution_time, tps_time, cpu_usage, memory_usage, gpu_usages, gpu_memory_usages, response, history_file):
    history_file.write(f">>> Iteration {i + 1} <<<\n")
    history_file.write(f"Execution Time  : {execution_time:.2f} seconds\n")
    history_file.write(f"Token Per Seconds Time  : {tps_time:.2f} \n")
    history_file.write(f"CPU Usage       : {cpu_usage:.2f} %\n")
    history_file.write(f"Memory Usage    : {memory_usage:.2f} MB\n")
    history_file.write(f"GPU Usage       : {gpu_usages} %\n")
    history_file.write(f"GPU Memory Usage: {gpu_memory_usages} MB\n")
    history_file.write(f"LLM Response    : {response}\n")
    history_file.write("-" * 40 + "\n\n")

def write_summary_report(report_file_name, model_name, question, averages, max_results, min_results):
    with open(report_file_name, "w", encoding="utf8") as report_file:
        report_file.write(f"file_name: {report_file_name}\n")
        report_file.write("=== Performance Measurement Report ===\n\n")
        report_file.write(f"model_name: {model_name}\n")
        report_file.write(f"question: {question}\n\n")

        report_file.write(">>> Average Results <<<\n")
        report_file.write(f"Average Execution Time  : {averages['execution_time']:.2f} seconds\n")
        report_file.write(f"Average Token Per Seconds Time  : {averages['tps_time']:.2f} \n")
        report_file.write(f"Average CPU Usage       : {averages['cpu_usage']:.2f} %\n")
        report_file.write(f"Average Memory Usage    : {averages['memory_usage']:.2f} MB\n")
        report_file.write(f"Average GPU Usage       : {averages['gpu_usages']} %\n")
        report_file.write(f"Average GPU Memory Usage: {averages['gpu_memory_usages']} MB\n")
        report_file.write("-" * 40 + "\n\n")

        report_file.write(">>> Maximum Execution Time <<<\n")
        report_file.write(f"Execution Time  : {max_results['execution_time']:.2f} seconds (Iteration {max_results['index'] + 1})\n")
        report_file.write(f"Token Per Seconds Time  : {max_results['tps_time']:.2f} (Iteration {max_results['index'] + 1})\n")
        report_file.write(f"CPU Usage       : {max_results['cpu_usage']:.2f} %\n")
        report_file.write(f"Memory Usage    : {max_results['memory_usage']:.2f} MB\n")
        report_file.write(f"GPU Usage       : {max_results['gpu_usages']} %\n")
        report_file.write(f"GPU Memory Usage: {max_results['gpu_memory_usages']} MB\n")
        report_file.write("-" * 40 + "\n\n")

        report_file.write(">>> Minimum Execution Time <<<\n")
        report_file.write(f"Execution Time  : {min_results['execution_time']:.2f} seconds (Iteration {min_results['index'] + 1})\n")
        report_file.write(f"Token Per Seconds Time  : {min_results['tps_time']:.2f} (Iteration {min_results['index'] + 1})\n")
        report_file.write(f"CPU Usage       : {min_results['cpu_usage']:.2f} %\n")
        report_file.write(f"Memory Usage    : {min_results['memory_usage']:.2f} MB\n")
        report_file.write(f"GPU Usage       : {min_results['gpu_usages']} %\n")
        report_file.write(f"GPU Memory Usage: {min_results['gpu_memory_usages']} MB\n")
        report_file.write("-" * 40 + "\n\n")

        report_file.write("=== End of Report ===\n")


def export_report(file_name, 
                  model_name,
                  num_iterations,            
                  execution_times,
                  tps_times,
                  cpu_usages,
                  memory_usages,
                  gpu_usages_list,
                  gpu_memory_usages_list,
                  responses,):

    gpu_name = get_gpu_name()
    cpu_name = get_cpu_name()
    report_file_name  = f"{file_name}_test_report_{gpu_name}_{cpu_name}.txt"
    history_file_name = f"{file_name}_test_history_{gpu_name}_{cpu_name}.txt"
    
    with open(history_file_name, "w", encoding="utf8") as history_file:
        history_file.write(f"file_name: {history_file_name}\n")
        history_file.write("=== Performance Measurement Report ===\n\n")
        for i, execution_time, tps_time, cpu_usage, memory_usage, gpu_usages, gpu_memory_usages, response in zip(
            range(num_iterations),
            execution_times,
            tps_times,
            cpu_usages,
            memory_usages,
            gpu_usages_list,
            gpu_memory_usages_list,
            responses,
        ):
            log_iteration_results(i, execution_time, tps_time, cpu_usage, memory_usage, gpu_usages, gpu_memory_usages, response, history_file)
            print(f"Iteration {i+1}/{num_iterations} completed.")
        
        history_file.write("=== End of History ===\n")

    
    averages = {
        'execution_time': sum(execution_times) / num_iterations,
        'tps_time': sum(tps_times) / num_iterations,
        'cpu_usage': sum(cpu_usages) / num_iterations,
        'memory_usage': sum(memory_usages) / num_iterations,
        'gpu_usages': [sum(gpu_usages) / num_iterations for gpu_usages in zip(*gpu_usages_list)],
        'gpu_memory_usages': [sum(gpu_mem) / num_iterations for gpu_mem in zip(*gpu_memory_usages_list)]
    }
    
    max_index = execution_times.index(max(execution_times))
    max_results = {
        'execution_time': execution_times[max_index],
        'tps_time': tps_times[max_index],
        'cpu_usage': cpu_usages[max_index],
        'memory_usage': memory_usages[max_index],
        'gpu_usages': gpu_usages_list[max_index],
        'gpu_memory_usages': gpu_memory_usages_list[max_index],
        'index': max_index
    }
    
    min_index = execution_times.index(min(execution_times))
    min_results = {
        'execution_time': execution_times[min_index],
        'tps_time': tps_times[min_index],
        'cpu_usage': cpu_usages[min_index],
        'memory_usage': memory_usages[min_index],
        'gpu_usages': gpu_usages_list[min_index],
        'gpu_memory_usages': gpu_memory_usages_list[min_index],
        'index': min_index
    }
    
    write_summary_report(report_file_name, model_name, question, averages, max_results, min_results)
    print(f"Report output saved to {report_file_name}")

# langchain_community

In [4]:
from langchain_community.llms import Ollama

In [15]:
model_name = 'llama3.1:8b'
# model_name = 'llama3.1'
file_name = "langchain"

llm = Ollama(model=model_name, temperature=0, top_p=0.3)
question = """
于某，男，62岁。患冠心病两年，服西药治疗，一日三次，从未有断，然胸憋心悸，一直不止。近月余，
每至夜则咳嗽哮喘，痰涎清稀如水，倚息不能平卧，胸憋心悸尤甚。白昼则症状减轻。
询知腰脊酸困，背畏风寒，时眩晕，手足心微热，口渴欲饮，但不多饮，亦不思冷，纳便尚可，舌尖略红，苔白腻，脉沉缓。给出中医诊断和处方建议
"""

execution_times = []
tps_times = []
cpu_usages = []
memory_usages = []
gpu_usages_list = []
gpu_memory_usages_list = []
responses = []

num_iterations = 10
for i in tqdm(range(num_iterations)):
    process = psutil.Process()
    gpus = GPUtil.getGPUs()
    start_time = time.time()

    cpu_start = process.cpu_percent(interval=None)
    memory_start = process.memory_info().rss

    ## 計算 token per seconds
    is_first_output = False
    token_list = []
    first_time = None
    for token in llm.stream(question):
        # print(token)
        if not is_first_output:
            is_first_output= True
            first_time = time.time() - start_time
        token_list.append(token)
    response = "".join(token_list)
    execution_time = time.time() - start_time
    tps_time = len(token_list) / (execution_time - first_time)
   
    cpu_end = process.cpu_percent(interval=None)
    memory_end = process.memory_info().rss
 
    cpu_usage = cpu_end - cpu_start
    memory_usage = (memory_end - memory_start) / (1024 * 1024)
     
    gpu_usages = [gpu.load * 100 for gpu in gpus]
    gpu_memory_usages = [gpu.memoryUsed for gpu in gpus]

    execution_times.append(execution_time)
    tps_times.append(tps_time)
    cpu_usages.append(cpu_usage)
    memory_usages.append(memory_usage)
    gpu_usages_list.append(gpu_usages)
    gpu_memory_usages_list.append(gpu_memory_usages)
    responses.append(response)

export_report(file_name, 
                  model_name,
                  num_iterations,            
                  execution_times,
                  tps_times,
                  cpu_usages,
                  memory_usages,
                  gpu_usages_list,
                  gpu_memory_usages_list,
                  responses)

  0%|          | 0/10 [00:00<?, ?it/s]

根据
患者
的
症
状
描述
，我
给
出的
中
医
诊
断
是
：


**
心
阳
虚
证
**


**
证
名
解
释
：
**

心
阳
虚
证
是
指
由于
长
期
服
用
西
药
治疗
冠
心
病
，
导致
心
阳
不足
，
出现
胸
憋
、
心
悸
、
眩
晕
等
症
状
。


**
处
方
建议
：
**


**
方
名
：
补
心
汤
加
减
**


**
组
成
：
**


*
  
 **
人
参
**
 
12
克


*
  
 **
白
术
**
 
15
克


*
  
 **
甘
草
**
 
6
克


*
  
 **
当
归
**
 
9
克


*
  
 **
川
楝
子
**
 
9
克


*
  
 **
枳
实
**
 
10
克


*
  
 **
麻
黄
**
 
3
克


*
  
 **
苍
术
**
 
10
克


*
  
 **
大
枣
**
 
15
克



**
用
法
：
**


*
  
 每
日
一
剂
，
热
水
送
服
。

*
  
 加
减
根据
患者
的
具体
情况
调整
。


**
注意
事
项
：
**


*
  
 患
者
应
避
免
过
度
劳
累
和
精神
紧
张
。

*
  
 鼓
励
患者
进行
适
当
的
运动
，如
散
步
、
瑜
伽
等
，以
改
善
心
血
管
健康
。

*
  
 患
者
应
定
期
检查
心
脏
功能
，
及
时
调整
治疗
方案
。


**
预
防
措施
：
**


*
  
 患
者
应
避
免
长
时间
屏
息
或
屏
气
，以
防
止
心
率
过
快
。

*
  
 鼓
励
患者
进行
深
呼
吸
和
放
松
技
巧
，以
缓
解
焦
虑
和
压
力
。

*
  
 患
者
应
定
期
检查
血
压
和
心
率
，
及
时
调整
治疗
方案
。


**
预
后
：
**


*
  
 根
据
患者
的
具体
情况
，
补
心
汤
加
减
可以
有效
改
善
心
阳
虚
证
的
症
状
。

*
  


 10%|█         | 1/10 [00:06<00:58,  6.50s/it]

 患
者
应
配合
医
生
进行
长
期
随
访
，以
确
保
治疗
效果
。

根据
患者
的
症
状
描述
，我
给
出的
中
医
诊
断
是
：


**
心
阳
虚
证
**


**
证
名
解
释
：
**

心
阳
虚
证
是
指
由于
长
期
服
用
西
药
治疗
冠
心
病
，
导致
心
阳
不足
，
出现
胸
憋
、
心
悸
、
眩
晕
等
症
状
。


**
处
方
建议
：
**


**
方
名
：
补
心
养
阴
汤
加
减
**


**
组
成
：
**


*
  
 **
人
参
**
 
12
克


*
  
 **
白
术
**
 
15
克


*
  
 **
甘
草
**
 
6
克


*
  
 **
枳
壳
**
 
9
克


*
  
 **
当
归
**
 
9
克


*
  
 **
川
楝
子
**
 
9
克


*
  
 **
麻
黄
**
 
3
克


*
  
 **
苍
术
**
 
12
克


*
  
 **
生
地
**
 
15
克


*
  
 **
天
冬
**
 
15
克


*
  
 **
麦
门
冬
**
 
15
克


*
  
 **
甘
遂
**
 
6
克


*
  
 **
枳
实
**
 
9
克



**
用
法
：
**


*
  
 每
日
一
剂
，
水
煎
取
汁
，
分
三
次
服
用
。

*
  
 加
减
根据
患者
的
具体
情况
调整
。


**
注意
事
项
：
**


*
  
 患
者
应
避
免
过
度
劳
累
和
精神
紧
张
。

*
  
 鼓
励
患者
进行
适
当
的
运动
，如
散
步
、
瑜
伽
等
，以
改
善
心
血
管
健康
。

*
  
 患
者
应
保持
良
好的
睡
眠
习
惯
，
避
免
夜
间
咳
嗽
哮
喘
。


**
预
防
措施
：
**


*
  
 定
期
检查
心
脏
功能
和
肺
部
健康
。

*
  
 避
免
吸
烟
和
暴
露
于
空
气
污
染
中
。

*
  
 鼓
励
患者


 20%|██        | 2/10 [00:10<00:41,  5.22s/it]

...

炼
，以
改
善
心
血
管
健康
。

根据
患者
的
症
状
描述
，我
给
出的
中
医
诊
断
是
：


**
心
阳
虚
证
**


**
证
名
解
释
：
**

心
阳
虚
证
是
指
由于
长
期
服
用
西
药
治疗
冠
心
病
，
导致
心
阳
不足
，
出现
胸
憋
、
心
悸
、
眩
晕
等
症
状
。


**
处
方
建议
：
**


**
方
名
：
补
心
养
阴
汤
加
减
**


**
组
成
：
**


*
  
 **
人
参
**
 
12
克


*
  
 **
白
术
**
 
15
克


*
  
 **
甘
草
**
 
6
克


*
  
 **
枳
壳
**
 
9
克


*
  
 **
当
归
**
 
9
克


*
  
 **
川
楝
子
**
 
9
克


*
  
 **
麻
黄
**
 
3
克


*
  
 **
苍
术
**
 
12
克


*
  
 **
生
地
**
 
15
克


*
  
 **
天
冬
**
 
15
克


*
  
 **
麦
门
冬
**
 
15
克


*
  
 **
甘
遂
**
 
6
克


*
  
 **
枳
实
**
 
9
克



**
用
法
：
**


*
  
 每
日
一
剂
，
水
煎
取
汁
，
分
三
次
服
用
。

*
  
 加
减
根据
患者
的
具体
情况
调整
。


**
注意
事
项
：
**


*
  
 患
者
应
避
免
过
度
劳
累
和
精神
紧
张
。

*
  
 鼓
励
患者
进行
适
当
的
运动
，如
散
步
、
瑜
伽
等
，以
改
善
心
血
管
健康
。

*
  
 患
者
应
保持
良
好的
睡
眠
习
惯
，
避
免
夜
间
咳
嗽
哮
喘
。


**
预
防
措施
：
**


*
  
 定
期
检查
心
脏
功能
和
肺
部
健康
。

*
  
 避
免
吸
烟
和
暴
露
于
空
气
污
染
中
。

*
  
 鼓
励
患者
进行
适
当
的
运动
和
锻


 90%|█████████ | 9/10 [00:40<00:04,  4.33s/it]

炼
，以
改
善
心
血
管
健康
。

根据
患者
的
症
状
描述
，我
给
出的
中
医
诊
断
是
：


**
心
阳
虚
证
**


**
证
名
解
释
：
**

心
阳
虚
证
是
指
由于
长
期
服
用
西
药
治疗
冠
心
病
，
导致
心
阳
不足
，
出现
胸
憋
、
心
悸
、
眩
晕
等
症
状
。


**
处
方
建议
：
**


**
方
名
：
补
心
养
阴
汤
加
减
**


**
组
成
：
**


*
  
 **
人
参
**
 
12
克


*
  
 **
白
术
**
 
15
克


*
  
 **
甘
草
**
 
6
克


*
  
 **
枳
壳
**
 
9
克


*
  
 **
当
归
**
 
9
克


*
  
 **
川
楝
子
**
 
9
克


*
  
 **
麻
黄
**
 
3
克


*
  
 **
苍
术
**
 
12
克


*
  
 **
生
地
**
 
15
克


*
  
 **
天
冬
**
 
15
克


*
  
 **
麦
门
冬
**
 
15
克


*
  
 **
甘
遂
**
 
6
克


*
  
 **
枳
实
**
 
9
克



**
用
法
：
**


*
  
 每
日
一
剂
，
水
煎
取
汁
，
分
三
次
服
用
。

*
  
 加
减
根据
患者
的
具体
情况
调整
。


**
注意
事
项
：
**


*
  
 患
者
应
避
免
过
度
劳
累
和
精神
紧
张
。

*
  
 鼓
励
患者
进行
适
当
的
运动
，如
散
步
、
瑜
伽
等
，以
改
善
心
血
管
健康
。

*
  
 患
者
应
保持
良
好的
睡
眠
习
惯
，
避
免
夜
间
咳
嗽
哮
喘
。


**
预
防
措施
：
**


*
  
 定
期
检查
心
脏
功能
和
肺
部
健康
。

*
  
 避
免
吸
烟
和
暴
露
于
空
气
污
染
中
。

*
  
 鼓
励
患者
进行
适
当
的
运动
和
锻


100%|██████████| 10/10 [00:45<00:00,  4.51s/it]

炼
，以
改
善
心
血
管
健康
。

Iteration 1/10 completed.
Iteration 2/10 completed.
Iteration 3/10 completed.
Iteration 4/10 completed.
Iteration 5/10 completed.
Iteration 6/10 completed.
Iteration 7/10 completed.
Iteration 8/10 completed.
Iteration 9/10 completed.
Iteration 10/10 completed.
Report output saved to langchain_test_report_NVIDIA GeForce RTX 4090_i9_14900.txt


# ollama

In [6]:
import ollama

In [ ]:
ollama_client = ollama.Client(host='http://192.168.1.246:11434')

In [8]:
ollama_client = ollama.Client(host='http://localhost:11434')

In [19]:
model_name = 'llama3.1:8b'
# model_name = 'llama3.1'
file_name = "ollama"

question = """
于某，男，62岁。患冠心病两年，服西药治疗，一日三次，从未有断，然胸憋心悸，一直不止。近月余，
每至夜则咳嗽哮喘，痰涎清稀如水，倚息不能平卧，胸憋心悸尤甚。白昼则症状减轻。
询知腰脊酸困，背畏风寒，时眩晕，手足心微热，口渴欲饮，但不多饮，亦不思冷，纳便尚可，舌尖略红，苔白腻，脉沉缓。给出中医诊断和处方建议
"""

execution_times = []
tps_times = []
cpu_usages = []
memory_usages = []
gpu_usages_list = []
gpu_memory_usages_list = []
responses = []

num_iterations = 10
for i in tqdm(range(num_iterations)):
    process = psutil.Process()
    gpus = GPUtil.getGPUs()
    start_time = time.time()

    cpu_start = process.cpu_percent(interval=None)
    memory_start = process.memory_info().rss

    ## 計算 token per seconds
    is_first_output = False
    token_list = []
    first_time = None
    for token in ollama_client.generate(model=model_name, prompt=question, stream=True,options={"temperature":0}):
        if not is_first_output:
            is_first_output= True
            first_time = time.time() - start_time
        token_list.append(token["response"])
    response = "".join(token_list)
    execution_time = time.time() - start_time
    tps_time = len(token_list) / (execution_time - first_time)
   
    cpu_end = process.cpu_percent(interval=None)
    memory_end = process.memory_info().rss
 
    cpu_usage = cpu_end - cpu_start
    memory_usage = (memory_end - memory_start) / (1024 * 1024)
     
    gpu_usages = [gpu.load * 100 for gpu in gpus]
    gpu_memory_usages = [gpu.memoryUsed for gpu in gpus]

    execution_times.append(execution_time)
    tps_times.append(tps_time)
    cpu_usages.append(cpu_usage)
    memory_usages.append(memory_usage)
    gpu_usages_list.append(gpu_usages)
    gpu_memory_usages_list.append(gpu_memory_usages)
    responses.append(response)

export_report(file_name, 
                  model_name,
                  num_iterations,            
                  execution_times,
                  tps_times,
                  cpu_usages,
                  memory_usages,
                  gpu_usages_list,
                  gpu_memory_usages_list,
                  responses)

100%|██████████| 10/10 [00:46<00:00,  4.68s/it]

Iteration 1/10 completed.
Iteration 2/10 completed.
Iteration 3/10 completed.
Iteration 4/10 completed.
Iteration 5/10 completed.
Iteration 6/10 completed.
Iteration 7/10 completed.
Iteration 8/10 completed.
Iteration 9/10 completed.
Iteration 10/10 completed.
Report output saved to ollama_test_report_NVIDIA GeForce RTX 4090_i9_14900.txt
